In [ ]:
%matplotlib notebook

# This examples shows who to download files from the ONC server
import glob
import os

import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import strawb
import strawb.sensors.lidar
import strawb.tools

import plotly
import plotly.graph_objs as go
import plotly.express as px
import pandas
import plotly.io as pio
pio.templates.default = "plotly_white"

import random

plt.rcParams.update({"text.usetex": False})  # fix time fmt of x_ticks

In [ ]:
def get_working_file_randomly(file_list):
    """Some files fails with the file_handler, i.e., if a group is missing for older SDAQ versions.
    To get one file randomly which works, try it untill it works.
    """

    success = False
    file_list_trials = file_list.copy()
    while len(file_list_trials) > 0 and not success:
        i = np.random.randint(0,len(file_list_trials))
        full_path = file_list_trials[i]
        try:
            lidar = strawb.sensors.lidar.FileHandler(full_path)
        except Exception as a:
            file_list_trials.pop(i)
            print(f'Error: {full_path} failed with: {a}')
        else:
            print(f'Success: {full_path}')
            return full_path
    return None
        

In [ ]:
if True: # takes one file randomly, takes some seconds
    file_list = strawb.sensors.lidar.FileHandler.find_files('*-SDAQ-LIDAR.hdf5',
                                               directory=strawb.Config.raw_data_dir,
                                               recursive=True,
                                               raise_nothing_found=True)

    full_path = get_working_file_randomly(file_list)  # select a random file
else:  # faster
    full_path = '/dss/strawb/raw_module_data/tummuontracker001/old_files/sdaq/TUMMUONTRACKER001_20210806T000000.000Z_20210806T235959.999Z-SDAQ-MODULE.hdf5'
#file_name = os.path.split(full_path)[-1]

print(f'Use: {full_path}')

In [ ]:
lidar = strawb.sensors.lidar.FileHandler(full_path)
lidar.file_attributes  # show the file attributes, time in seconds since epoch (1.1.1970) UTC

# Set up plotting function

In [ ]:
def multiline_plot(data_frame):
    columns = list(set(data_frame.columns).difference(['time']))
    
    fig = plotly.subplots.make_subplots(shared_xaxes=True,
                                        rows=len(columns),
                                        cols=1, 
                                        #subplot_titles=columns
                                       )


    fig.update_layout(
        width=800,
        height=150+100*len(columns),)

    j = 1
    for i, column_i in enumerate(columns):
        fig.add_trace(go.Scatter(x=data_frame['time'], 
                                 y=data_frame[column_i],
                                 name=column_i), 
                      row=i+1, col=1
                     )

    return fig

# Plotting with pandas and plotly

In [ ]:
df_daq = pandas.DataFrame(dict(time = lidar.daq_time.asdatetime()[:],
                               frequency_pmt = lidar.daq_frequency_pmt,
                               frequency_trigger = lidar.daq_frequency_trigger,
                               pmt = lidar.daq_pmt,
                               pulser_readout = lidar.daq_pulser_readout,
                               pulser_trigger = lidar.daq_pulser_trigger,
                               state = lidar.daq_state,
                               trb = lidar.daq_trb))

In [ ]:
# Gimbal
df_gimabl = pandas.DataFrame(dict(time = lidar.gimbal_time.asdatetime()[:],
                                  delay = lidar.gimbal_delay,
                                  pos_x = lidar.gimbal_pos_x,
                                  pos_y = lidar.gimbal_pos_y,
                                  power = lidar.gimbal_power,)
                         )

In [ ]:
# Laser
df_laser = pandas.DataFrame(dict(time = lidar.laser_time.asdatetime()[:],
                                 diode = lidar.laser_diode,
                                 frequency = lidar.laser_frequency,
                                 power = lidar.laser_power,
                                 pulsewidth = lidar.laser_pulsewidth,
                                 set_adjust_x = lidar.laser_set_adjust_x,
                                 set_adjust_y = lidar.laser_set_adjust_y,)
                          )

# Basic plots

In [ ]:
fig = multiline_plot(df_daq)
fig.show()

In [ ]:
fig = multiline_plot(df_gimabl)
fig.show()

In [ ]:
fig = multiline_plot(df_laser)
fig.show()

# Load a file
## can be a single or combined aka. virtual HDF5 file

In [ ]:
# fig, ax = plt.subplots(nrows=8, squeeze=False, figsize=(7,15), sharex=True)
# ax = ax.flatten()

# i = 0

# # ACCEL
# strawb.tools.plot_binned_mean(module.accel_time, module.accel_x[:]*9.81, ax=ax[i], label='accel x')
# strawb.tools.plot_binned_mean(module.accel_time, module.accel_y[:]*9.81, ax=ax[i], label='accel y')
# ax[i].set_ylabel('Accel [m/s**2]')
# i += 1
# strawb.tools.plot_binned_mean(module.accel_time, module.accel_z[:]*9.81, ax=ax[i], label='accel z')
# ax[i].set_ylabel('Accel [m/s**2]')
# i += 1

# # Magnet
# strawb.tools.plot_binned_mean(module.magneto_time, module.magneto_x, ax=ax[i], label='magneto x')
# strawb.tools.plot_binned_mean(module.magneto_time, module.magneto_y, ax=ax[i], label='magneto y')
# strawb.tools.plot_binned_mean(module.magneto_time, module.magneto_z, ax=ax[i], label='magneto z')
# ax[i].set_ylabel('Magneto [a.u.]')
# i += 1

# # TEMP.
# strawb.tools.plot_binned_mean(module.temperatures_time, module.temperatures_temp1, ax=ax[i], label='temp 1')
# strawb.tools.plot_binned_mean(module.temperatures_time, module.temperatures_temp2, ax=ax[i], label='temp 2')
# ax[i].set_ylabel('Temp [C]')
# i += 1
# strawb.tools.plot_binned_mean(module.temperatures_time, module.temperatures_temp3, ax=ax[i], label='temp 3')
# ax[i].set_ylabel('Temp [C]')
# i += 1

# # PTH
# strawb.tools.plot_binned_mean(module.pth_time, module.pth_temperature, ax=ax[i], label='temp pth')
# ax[i].set_ylabel('Temp [C]')
# i += 1

# strawb.tools.plot_binned_mean(module.pth_time, module.pth_humidity, ax=ax[i], label='pth humidity')
# ax[i].set_ylabel('humidity [per]')
# i += 1

# strawb.tools.plot_binned_mean(module.pth_time, module.pth_pressure, ax=ax[i], label='pth pressure')
# ax[i].set_ylabel('pressure [mbar]')
# i += 1

# for ax_i in ax:
#     ax_i.legend(loc=1)
#     ax_i.grid()

# # # set limits
# # limits = np.array([[ax_i.dataLim.x0, ax_i.dataLim.x1] for ax_i in ax])
# # ax[-1].set_xlim((limits[:,0].min(), limits[:,1].max()))
    
# fig.autofmt_xdate()
# locator = mdates.AutoDateLocator()
# #fmt = mdates.AutoDateFormatter(locator, usetex=True)
# fmt = mdates.ConciseDateFormatter(locator)
# ax[-1].xaxis.set_major_formatter(fmt)
# plt.tight_layout()

In [ ]:
# limits = np.array([[ax_i.dataLim.x0, ax_i.dataLim.x1] for ax_i in ax])
# limits

## plot multiple modules (currently doesn't work: needs viratul hdf5 files or a file per moduel)

In [ ]:
# module_list = []

# for dev_i in strawb.dev_codes_deployed:
#     # get a list with all filenames that should be included in the virtual HDF5 file
#     f_names = glob.glob(f'{strawb.Config.raw_data_dir}/{dev_i}_*MODULE.hdf5')
#     f_names.sort()  # f_names should be sorted

#     # create the virtual HDF5 file, f_names should be sorted, otherwise the data isn't sorted
#     virtual_hdf5_gen = strawb.VirtualHDF5(f'{dev_i}_MODULE.h5',
#                                           f_names,
#                                           obj_dict_filter=strawb.DatasetsInGroupSameSize)

#     module_list.append(strawb.sensors.module.FileHandler(f'{dev_i}_MODULE.h5'))

In [ ]:
# fig, ax = plt.subplots(nrows=6, squeeze=False, figsize=(10,15), sharex=True)
# ax = ax.flatten()


# for module in module_list:
#     print(module.module)
#     i = 0
#     # TEMP.
#     mask = module.temperatures_time[:] > datetime.datetime(year=2020,month=12,day=1).timestamp()
#     plot_time = module.temperatures_time[:][mask]
#     strawb.tools.plot_binned_mean(plot_time, module.temperatures_temp1[:][mask],
#                                   ax=ax[i], label=f'temp 1 {module.module}')
#     ax[i].set_ylabel('Temp [C]')
#     i += 1

#     strawb.tools.plot_binned_mean(plot_time, module.temperatures_temp2[:][mask],
#                                   ax=ax[i], label=f'temp 2 {module.module}')
#     ax[i].set_ylabel('Temp [C]')
#     i += 1

#     strawb.tools.plot_binned_mean(plot_time, module.temperatures_temp3[:][mask],
#                                   ax=ax[i], label=f'temp 3 {module.module}')
#     ax[i].set_ylabel('Temp [C]')
#     i += 1

#     # PTH
#     mask = module.pth_time[:] > datetime.datetime(year=2020,month=12,day=1).timestamp()
#     plot_time = module.pth_time[:][mask]
#     strawb.tools.plot_binned_mean(plot_time, module.pth_temperature[:][mask],
#                                   ax=ax[i], label=f'temp pth {module.module}')
#     ax[i].set_ylabel('Temp [C]')
#     i += 1

#     strawb.tools.plot_binned_mean(plot_time, module.pth_humidity[:][mask],
#                                   ax=ax[i], label=f'pth humidity {module.module}')
#     ax[i].set_ylabel('humidity [per]')
#     i += 1

#     strawb.tools.plot_binned_mean(plot_time, module.pth_pressure[:][mask],
#                                   ax=ax[i], label=f'pth pressure {module.module}')
#     ax[i].set_ylabel('pressure [mbar]')
#     i += 1

# for ax_i in ax:
#     ax_i.legend(loc=0)
#     ax_i.grid()

# # set limits
# limits = np.array([[ax_i.dataLim.x0, ax_i.dataLim.x1] for ax_i in ax])
# ax[-1].set_xlim((limits[:,0].min(), limits[:,1].max()))
    
# fig.autofmt_xdate()
# locator = mdates.AutoDateLocator()
# #fmt = mdates.AutoDateFormatter(locator, usetex=True)
# fmt = mdates.ConciseDateFormatter(locator)
# ax[-1].xaxis.set_major_formatter(fmt)
# plt.tight_layout()